### Student Information
Name: 張筱萍

Student ID: `111065529`

GitHub ID: `hsiaoping-nthu`

Kaggle name: `hsiaoping(?)`

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__. 

In [2]:
### Begin Assignment Here
import pandas as pd
import numpy as np

In [10]:
import json

# # # read json # # #
data = []
hashtags, tweet_ids, texts = [], [], []  # date information ?
with open('dm2022-isa5810-lab2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        item = json.loads(line)['_source']['tweet']
        hashtags.append(item['hashtags'])
        tweet_ids.append(item['tweet_id'])
        texts.append(item['text'])

# convert to dataframe and export to csv file
df = pd.DataFrame(data={'hashtag': hashtags, 'tweet_id': tweet_ids, 'text': texts})
df.to_csv('tweet_data.csv', index=False)
df.head()

finish


,hashtag,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [4]:
## read 
identification = pd.read_csv('dm2022-isa5810-lab2-homework/data_identification.csv')
identification.head()

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [5]:
emotion = pd.read_csv('dm2022-isa5810-lab2-homework/emotion.csv')
emotion.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [6]:
# merge df with tweet_id
temp_df = pd.merge(identification, emotion)
temp_df.head()

,tweet_id,identification,emotion
0,0x29e452,train,joy
1,0x2b3819,train,joy
2,0x2a2acc,train,trust
3,0x2a8830,train,joy
4,0x20b21d,train,anticipation


In [11]:
all_df = pd.merge(df, temp_df)
all_df.head()

,hashtag,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation


In [12]:
all_df.to_csv('tweet_data.csv', index=False)
print('finish')

finish


### load preprocessing data

In [15]:
data = pd.read_csv('tweet_data.csv', lineterminator='\n')
data.head()

,hashtag,tweet_id,text,identification,emotion
0,['Snapchat'],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"['freepress', 'TrumpLegacy', 'CNN']",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,"['authentic', 'LaughOutLoud']",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation
